In [1]:
import json

# Load the JSON file
with open('C://Users//Jayashrinidhi V//OneDrive//Documents//VScode//Justice_bot//doj_website_data.json') as file:
    data = json.load(file)

# Initialize lists for training data
training_sentences = []
training_labels = []
labels = []

# Process the JSON data
for item in data:
    if isinstance(item, dict):  # Check if the item is a dictionary
        # Extract fields if they exist
        page_content = item.get('page_content', '')
        if page_content:
            training_sentences.append(page_content)
            training_labels.append('page_content')  # Label for the content

        link_text = item.get('link_text', '')
        if link_text:
            training_sentences.append(link_text)
            training_labels.append('link_text')  # Label for the link text

        url = item.get('url', '')
        if url:
            training_sentences.append(url)
            training_labels.append('url')  # Label for the URL
    else:
        print(f"Skipping non-dict item: {item}")

# Continue with the rest of your pipeline...


Skipping non-dict item: text_content
Skipping non-dict item: links
Skipping non-dict item: headers


In [2]:
import json

# Load the JSON file
with open('C://Users//Jayashrinidhi V//OneDrive//Documents//VScode//Justice_bot//doj_full_scrape.json') as file:
    data = json.load(file)


In [3]:
import json

# Load the JSON file
with open('C://Users//Jayashrinidhi V//OneDrive//Documents//VScode//Justice_bot//doj_full_scrape.json') as file:
    data = json.load(file)

# Check the type of data
print(type(data))  # This will tell you whether data is a dict, list, etc.

# If it's a dictionary, print its keys
if isinstance(data, dict):
    print(data.keys())  # Print the keys to see the structure

# If it's a list, you can then proceed with printing the first few items
elif isinstance(data, list):
    print(data[:5])  # Print the first 5 items



<class 'list'>
[{'link_text': '\nSkip to main content\n\n', 'url': 'https://doj.gov.in/#SkipContent', 'page_content': 'As per the Allocation of Business (Rules), 1961, Department of Justice is a part of Ministry of Law & Justice, Government of India. It is one of the oldest Ministries of the Government of India. Till 31.12.2009, Department of Justice was part of Ministry of Home Affairs and Union Home Secretary had been the Secretary of Department of Justice. Keeping in view the increasing workload and formulating many policies and programmes on Judicial Reforms in the country, a separate Department namely Department of Justice was carved out from MHA and placed under the charge of Secretary to Government of India and it started working as such from 1st January, 2010 under the Ministry of Law & Justice. The Department is housed in the Jaisalmer House, 26, Man Singh Road, New Delhi. The Organizational setup of the Department includes 04 Joint Secretaries, 08 Directors/ Deputy Secretarie

In [4]:
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Load the JSON file
with open('C://Users//Jayashrinidhi V//OneDrive//Documents//VScode//Justice_bot//doj_website_data.json') as file:
    data = json.load(file)

# Initialize lists for training data
training_sentences = []
training_labels = []
labels = []

# Process the JSON data
for item in data:
    # Assuming each item in the list is a dictionary with 'link_text', 'url', 'page_content'
    page_content = item.get('page_content', '')
    if page_content:
        training_sentences.append(page_content)
        training_labels.append('page_content')  # Label for the content
    
    # You can also include 'link_text' and 'url' if needed
    link_text = item.get('link_text', '')
    if link_text:
        training_sentences.append(link_text)
        training_labels.append('link_text')  # Label for the link text
    
    url = item.get('url', '')
    if url:
        training_sentences.append(url)
        training_labels.append('url')  # Label for the URL

# Update labels list if using label encoding
for label in ['page_content', 'link_text', 'url']:
    if label not in labels:
        labels.append(label)

# Encode the labels
label_encoder = LabelEncoder()
training_labels = label_encoder.fit_transform(training_labels)

# Tokenize the sentences
vocab_size = 1000  # Adjust as necessary
max_length = 200  # Adjust based on your data
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Convert labels to categorical format if using a neural network
training_labels = to_categorical(training_labels, num_classes=len(labels))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, training_labels, test_size=0.2, random_state=42)

# Now you can proceed with building, training, and evaluating your model


AttributeError: 'str' object has no attribute 'get'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(len(labels), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model architecture
model.summary()


NameError: name 'vocab_size' is not defined

In [ ]:
# Train the model
history = model.fit(
    X_train, 
    y_train, 
    epochs=10,          # Adjust epochs based on your needs
    validation_data=(X_test, y_test),
    batch_size=32,      # Adjust batch size as necessary
    verbose=2           # Display training progress
)


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Convert predictions back to label format
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))

# Print some sample predictions
for i in range(5):
    print(f'Input: {X_test[i]}')
    print(f'Predicted Label: {predicted_labels[i]}')
    print(f'Actual Label: {label_encoder.inverse_transform([y_test[i].argmax()])[0]}')
    print('---')
